In [29]:
import pandas as pd
import numpy as np

In [30]:
###### 1 Min Data - 1 Step Forecasting ######

In [31]:
def CreateData(data_c, shif, train_index):
    data_c.drop("system_time", axis = 1)
    data_c['Next_mid'] = data_c.midpoint.shift(-shif)
    data_c['Price_move'] = (data_c.Next_mid - data_c.midpoint) / data_c.midpoint
    data_c.dropna(axis=0, inplace=True) #drop the last row which is now Nan

    data_c['y'] = 0
    data_c.loc[(data_c['Price_move'] < -0.001), 'y'] = 1
    data_c.loc[(data_c['Price_move'] > 0.001), 'y'] = 2
    data_c.drop(['Next_mid', 'midpoint', 'Price_move', "system_time", "Return"], axis=1, inplace=True)
    Y = data_c.pop("y")
    
    data_std = data_c

    r, x = data_c.shape

    for j in range(x):
        m = data_c.iloc[:train_index,j].mean()
        s = data_c.iloc[:train_index,j].std()
        for i in range(r):
            data_std.iloc[i,j] = (data_c.iloc[i,j] - m) / s
    
    return data_std, Y

In [32]:
def CreateBidAskData(data_c):
    bids = data_c.iloc[:,0:18]
    asks = data_c.iloc[:,78:93]
    data_bid_ask  = pd.concat([bids.reset_index(drop=True), asks], axis=1)
    
    return data_bid_ask
    

In [33]:
def CreateOderFlowData(data_c):
    data_order_flow = data_c.iloc[:,:153]
    
    return(data_order_flow)

In [34]:
data_1min = pd.read_csv("data/final/1min/BTC.csv")
data_5min = pd.read_csv("data/final/5min/BTC.csv")

In [35]:
data_5min

,system_time,midpoint,spread,buys,sells,bids_distance_0,bids_distance_1,bids_distance_2,bids_distance_3,bids_distance_4,...,RK,PreAvg,JV,PJ,SV,ASV,RQ,RQTri,RQQuad,NV
0,2021-04-07 12:52:41.122161+00:00,56440.805,0.01,5.171417e+06,2.174014e+06,-8.858839e-08,-6.156893e-05,-1.473225e-04,-0.000357,-0.000358,...,54816.870101,-0.000003,0.000000,0.000000,3.995435e-06,0.000004,7.192014e-08,9.243683e-09,2.818324e-09,6131.632817
1,2021-04-07 12:57:41.122161+00:00,56553.735,0.01,3.691003e+06,1.786106e+06,-8.841149e-08,-2.652345e-07,-9.725264e-07,-0.000038,-0.000052,...,146805.013079,-0.000003,0.000000,0.000000,3.162972e-06,0.000005,7.393567e-08,9.210710e-09,3.035693e-09,6965.551013
2,2021-04-07 13:02:41.122161+00:00,56453.245,0.01,4.135456e+06,3.622263e+06,-8.856887e-08,-2.571154e-04,-2.647323e-04,-0.000265,-0.000272,...,211591.248911,-0.000003,0.000000,0.000000,5.148534e-06,0.000005,7.321718e-08,1.009486e-08,3.248477e-09,7312.396113
3,2021-04-07 13:07:41.122161+00:00,56581.485,0.01,2.670256e+06,1.796536e+06,-8.836813e-08,-2.606860e-05,-2.624533e-05,-0.000037,-0.000037,...,167717.635532,-0.000003,0.000000,0.000000,2.431962e-07,0.000005,7.640872e-08,1.033797e-08,3.792979e-09,7224.027300
4,2021-04-07 13:12:41.122161+00:00,56609.395,0.01,2.285696e+06,8.870143e+05,-8.832456e-08,-2.649737e-07,-3.541815e-05,-0.000050,-0.000061,...,328190.278009,-0.000003,0.000000,0.000000,1.533561e-06,0.000004,5.972297e-08,8.121679e-09,3.583862e-09,7636.151300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3346,2021-04-19 09:25:00.386544+00:00,56875.045,0.01,3.338628e+05,3.432730e+05,-8.791202e-08,-3.138459e-05,-8.799993e-05,-0.000095,-0.000101,...,55980.964250,-0.000002,0.000000,0.000000,8.748612e-09,0.000003,4.156521e-08,5.683997e-09,1.127492e-09,5486.265889
3347,2021-04-19 09:30:00.386544+00:00,56880.365,0.01,4.450456e+05,3.306800e+05,-8.790379e-08,-9.309012e-05,-1.099677e-04,-0.000118,-0.000157,...,28701.456502,-0.000002,0.000000,0.000000,1.249401e-05,0.000002,2.371535e-08,3.445745e-09,9.973341e-10,5379.504242
3348,2021-04-19 09:35:00.386544+00:00,57081.775,0.01,4.093789e+05,2.487355e+05,-8.759363e-08,-1.138717e-06,-1.270108e-05,-0.000014,-0.000014,...,68519.049162,-0.000002,0.000002,0.068193,1.491607e-07,0.000003,3.059876e-08,1.179851e-09,8.073865e-10,3870.652804
3349,2021-04-19 09:40:00.386544+00:00,57103.825,0.01,1.766741e+06,1.765380e+06,-8.755981e-08,-6.768373e-05,-6.961005e-05,-0.000070,-0.000144,...,83009.739264,-0.000002,0.000000,0.000000,3.711875e-07,0.000002,3.043640e-08,1.155797e-09,5.129654e-10,4149.622242


In [36]:
data_5min.shape

(3351, 170)

In [37]:
3351/12

279.25

In [38]:
3350-280*2

2790

In [39]:
data_1min_std, Y_1min = CreateData(data_1min, 1, 14000)
data_1min = pd.read_csv("data/final/1min/BTC.csv")
data_5min = pd.read_csv("data/final/5min/BTC.csv")

data_1min = pd.read_csv("data/final/1min/BTC.csv")

data_1min_5_std, Y_1_5min = CreateData(data_1min, 5, 14000)

In [40]:
Y_1min.value_counts()

0    13901
1     1596
2     1551
Name: y, dtype: int64

In [20]:
def CreateData(data_c, shif, train_index):
    data_c.drop("system_time", axis = 1)
    data_c['Next_mid'] = data_c.midpoint.shift(-shif)
    data_c['Price_move'] = (data_c.Next_mid - data_c.midpoint) / data_c.midpoint
    data_c.dropna(axis=0, inplace=True) #drop the last row which is now Nan

    data_c['y'] = 0
    data_c.loc[(data_c['Price_move'] < -0.001), 'y'] = 1
    data_c.loc[(data_c['Price_move'] > 0.001), 'y'] = 2
    data_c.drop(['Next_mid', 'midpoint', 'Price_move', "system_time", "Return"], axis=1, inplace=True)
    Y = data_c.pop("y")
    
    data_std = data_c

    r, x = data_c.shape

    for j in range(x):
        m = data_c.iloc[:train_index,j].mean()
        s = data_c.iloc[:train_index,j].std()
        for i in range(r):
            data_std.iloc[i,j] = (data_c.iloc[i,j] - m) / s
    
    return data_std, Y

In [21]:
data_5min = pd.read_csv("data/final/5min/BTC.csv")
data_5min_std, Y_5min = CreateData(data_5min, 1, 2790)

In [23]:
Y_5min.value_counts()

0    1540
2     911
1     899
Name: y, dtype: int64

In [11]:
data_1min_std.to_csv("data/final/1min/HF/BTC_std.csv", index = False)
data_1min_5_std.to_csv("data/final/1min/HF/BTC_std_5.csv", index = False)

In [24]:
data_5min_std.to_csv("data/final/5min/HF/BTC_std.csv", index = False)

In [12]:
Y_1min.to_csv("data/final/1min/BTC_Y_std.csv", index = False)
Y_1_5min.to_csv("data/final/1min/BTC_Y_5_std.csv", index = False)


In [25]:
Y_5min.to_csv("data/final/5min/BTC_Y_std.csv", index = False)

In [18]:
OrderFlow1 = CreateOderFlowData(data_1min_std)
OrderFlow1_5 = CreateOderFlowData(data_1min_5_std)



OrderFlow1.to_csv("data/final/1min/OrderFlow/BTC_std.csv", index = False)
OrderFlow1_5.to_csv("data/final/1min/OrderFlow/BTC_std_5.csv", index = False)


In [26]:
OrderFlow5 = CreateOderFlowData(data_5min_std)
OrderFlow5.to_csv("data/final/5min/OrderFlow/BTC_std.csv", index = False)

In [19]:
BidAsk1 = CreateBidAskData(data_1min_std)
BidAsk1_5 = CreateBidAskData(data_1min_5_std)

BidAsk1.to_csv("data/final/1min/BidASk/BTC_std.csv", index = False)
BidAsk1_5.to_csv("data/final/1min/BidAsk/BTC_std_5.csv", index = False)


In [27]:
BidAsk5 = CreateBidAskData(data_5min_std)
BidAsk5.to_csv("data/final/5min/BidAsk/BTC_std.csv", index = False)